 # Лабораторная работа: От SVM к Ансамблям деревьев



 **Цель:** Практическое освоение ключевых алгоритмов машинного обучения: метода опорных векторов (SVM), деревьев решений (Decision Trees) и ансамблевых методов (Ensemble Learning). Основной фокус — на использовании моделей и понимании их логики работы.



 **План работы:**

 1.  Метод опорных векторов (SVM)

 2.  Деревья решений (Decision Trees)

 3.  Ансамблевые методы (Ensemble Learning)

 ## Часть 1: Метод опорных векторов (SVM)



 ### 1.1 Краткий обзор и интуиция



 **Принцип работы:** SVM ищет гиперплоскость, которая разделяет классы с максимальным отступом (маржой). Точки, ближайшие к этой гиперплоскости, называются опорными векторами и полностью определяют модель. Для нелинейно разделимых данных используется "ядерный трюк" — данные проецируются в пространство большей размерности, где они становятся линейно разделимыми.



 **Источники для изучения:**

 *   [Scikit-learn User Guide: Support Vector Machines](https://scikit-learn.org/stable/modules/svm.html)

 *   [Метод опорных векторов](https://habr.com/ru/companies/ods/articles/484148/?ysclid=mgrx76r29s638032555)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

# Генерируем данные
X, y = make_blobs(n_samples=50, centers=2, random_state=0, cluster_std=0.60) # type: ignore
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='autumn')
plt.title("Исходные данные")
plt.show()


 **Задание 1.1: Обучение линейного SVM**



 1.  Импортируйте класс `SVC` из `sklearn.svm`.

 2.  Создайте объект модели с линейным ядром (`kernel='linear'`) и параметром `C=100`.

 3.  Обучите модель на данных `X`, `y`.

 4.  Визуализируйте результат с помощью функции `plot_svc_decision_function` (она уже реализована ниже).

In [ ]:
from sklearn.svm import SVC

# TODO: Создайте и обучите модель
# model = ...

def plot_svc_decision_function(model, ax=None):
    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model.decision_function(xy).reshape(X.shape)
    ax.contour(X, Y, P, colors='k', levels=[-1, 0, 1], alpha=0.5, linestyles=['--', '-', '--'])
    ax.scatter(model.support_vectors_[:, 0], model.support_vectors_[:, 1], s=100, 
               linewidth=2, facecolors='none', edgecolors='black', label='Опорные векторы')
    ax.legend()

# Визуализация
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
# TODO: Раскомментируйте и вызовите функцию для вашей модели
# plot_svc_decision_function(model)
plt.title("SVM с линейным ядром")
plt.show()


 **Задание 1.2: Анализ атрибутов модели**



 Изучите атрибуты обученной модели `model`. Выведите на экран:

 1.  Координаты опорных векторов (`support_vectors_`).

 2.  Количество опорных векторов для каждого класса (`n_support_`).

In [ ]:
# TODO: Выведите координаты опорных векторов
# TODO: Выведите количество опорных векторов для каждого класса


 **Задание 1.3: Эксперименты с ядрами**



 **Принцип работы ядер:** Ядро — это функция, которая вычисляет скалярное произведение двух векторов в преобразованном пространстве признаков, не выполняя явного преобразования. Это позволяет эффективно работать с нелинейными зависимостями.



 Протестируйте разные ядра (`'linear'`, `'rbf'`, `'poly'`) на нелинейных датасетах (`make_moons`, `make_circles`). Проанализируйте, какое ядро лучше подходит для каждого типа данных.

In [ ]:
from sklearn.datasets import make_moons, make_circles

def test_kernels(X_data, y_data, title):
    kernels = ['linear', 'rbf', 'poly']
    plt.figure(figsize=(15, 4))
    for i, kernel in enumerate(kernels):
        plt.subplot(1, 3, i+1)
        # TODO: Создайте и обучите модель SVM с текущим ядром
        # clf = ...
        # clf.fit(...)
        plt.scatter(X_data[:, 0], X_data[:, 1], c=y_data, s=30, cmap='autumn')
        # plot_svc_decision_function(clf)
        plt.title(f'{title}, {kernel} kernel')
    plt.show()

# Генерация и тестирование на "Moons"
X_moon, y_moon = make_moons(n_samples=100, noise=0.1, random_state=0)
test_kernels(X_moon, y_moon, "Moons")

# Генерация и тестирование на "Circles"
X_circle, y_circle = make_circles(n_samples=100, noise=0.1, factor=0.2, random_state=0)
test_kernels(X_circle, y_circle, "Circles")


 ## Часть 2: Деревья решений (Decision Trees)



 ### 2.1 Краткий обзор и интуиция



 **Принцип работы:** Дерево решений строится рекурсивно, разбивая набор данных на подмножества на основе значений признаков. На каждом шаге выбирается такое разбиение, которое максимизирует "чистоту" дочерних узлов (минимизирует неоднородность). Для измерения неоднородности используются энтропия или индекс Джини. Процесс останавливается, когда достигается условие остановки (например, максимальная глубина или минимальное количество образцов в листе).



 **Источники для изучения:**

 *   [Scikit-learn User Guide: Decision Trees](https://scikit-learn.org/stable/modules/tree.html)

 *   [Хабр: Деревья решений — теория](https://habr.com/ru/companies/productstar/articles/523044/)

In [ ]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()
X, y = iris.data, iris.target # type: ignore
pd.DataFrame(iris.data, columns=iris.feature_names).head() # type: ignore


 **Задание 2.1: Обучение и визуализация дерева**



 1.  Импортируйте `DecisionTreeClassifier` из `sklearn.tree`.

 2.  Создайте объект дерева с критерием `'entropy'` и максимальной глубиной `3`.

 3.  Обучите дерево на данных `X`, `y`.

 4.  Выведите текстовое представление дерева с помощью `export_text`.

 5.  Визуализируйте дерево с помощью `plot_tree`.

In [ ]:
# TODO: Импортируйте необходимые модули

# TODO: Создайте и обучите дерево
# tree_clf = ...

# TODO: Выведите текстовое представление
# print(...)

# TODO: Визуализируйте дерево
# plt.figure(figsize=(15, 10))
# plot_tree(...)
# plt.show()


 **Задание 2.2: Реализация метрик неоднородности**



 Реализуйте функции для расчета энтропии и информационного выигрыша. Это поможет понять, как дерево принимает решения о разбиении.

In [ ]:
import numpy as np

def entropy(y):
    """Рассчитывает энтропию для массива меток."""
    # TODO: Реализуйте функцию
    # 1. Найдите уникальные классы и их количество с помощью np.unique(..., return_counts=True)
    # 2. Рассчитайте вероятности для каждого класса
    # 3. Вычислите энтропию по формуле: -sum(p * log2(p))
    # Не забудьте обработать случай, когда p=0 (log(0) не определён)
    pass

def information_gain(parent, left_child, right_child):
    """Рассчитывает информационный выигрыш от разбиения."""
    # TODO: Реализуйте функцию
    # IG = Entropy(parent) - (N_left/N_total * Entropy(left) + N_right/N_total * Entropy(right))
    pass

# Тест
y_parent = np.array([0]*50 + [1]*50)
y_left = np.array([0]*50)
y_right = np.array([1]*50)

print(f"Энтропия родителя: {entropy(y_parent):.3f}")
print(f"Информационный выигрыш: {information_gain(y_parent, y_left, y_right):.3f}")


 ## Часть 3: Ансамблевые методы (Ensemble Learning)



 ### 3.1 Краткий обзор и интуиция



 **Принцип работы:**

 *   **Бэггинг (Bagging):** Обучает множество моделей независимо на случайных подвыборках данных (с возвращением). Итоговое предсказание — это усреднение (регрессия) или голосование (классификация). Цель — уменьшить дисперсию модели.

 *   **Случайный лес (Random Forest):** Это бэггинг для деревьев решений с дополнительной случайностью: при каждом разбиении узла рассматривается только случайное подмножество признаков. Это снижает корреляцию между деревьями и улучшает обобщающую способность.

 *   **Бустинг (Boosting):** Обучает модели последовательно. Каждая новая модель пытается исправить ошибки предыдущих, уделяя больше внимания плохо классифицированным объектам (увеличивая их вес). Цель — уменьшить смещение модели.



 **Источники для изучения:**

 *   [Scikit-learn User Guide: Ensemble methods](https://scikit-learn.org/stable/modules/ensemble.html)

 *   [AdaBoost](https://neerc.ifmo.ru/wiki/index.php?title=Бустинг,_AdaBoost)

 *   [Ансамбли в машинном обучении](https://education.yandex.ru/handbook/ml/article/ansambli-v-mashinnom-obuchenii)

In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Загрузка и разбиение данных
digits = load_digits()
X, y = digits.data, digits.target # type: ignore
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


 **Задание 3.1: Сравнение одиночной модели и ансамблей**



 Обучите и сравните по точности:

 1.  Одиночное дерево решений.

 2.  Случайный лес (`RandomForestClassifier`).

 3.  AdaBoost (`AdaBoostClassifier`).



 Проанализируйте результаты.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
# TODO: Импортируйте RandomForestClassifier и AdaBoostClassifier

# 1. Одиночное дерево
# TODO: Создайте, обучите и оцените точность
single_tree_acc = 0.0

# 2. Случайный лес
# TODO: Создайте, обучите и оцените точность
# rf = 
rf_acc = 0.0

# 3. AdaBoost
# TODO: Создайте, обучите и оцените точность
# Используйте DecisionTreeClassifier как базовый классификатор
# ada = 
ada_acc = 0.0

print(f"Точность одиночного дерева: {single_tree_acc:.3f}")
print(f"Точность случайного леса: {rf_acc:.3f}")
print(f"Точность AdaBoost: {ada_acc:.3f}")


 **Задание 3.2: Бэггинг вручную**



 Реализуйте простой алгоритм бэггинга для деревьев решений. Это поможет глубже понять его принцип работы.

In [ ]:
import numpy as np
from scipy import stats

def bootstrap_sample(X, y):
    """Создаёт бутстрэп-выборку из данных X, y."""
    n_samples = X.shape[0]
    indices = np.random.choice(n_samples, size=n_samples, replace=True)
    return X[indices], y[indices]

n_trees = 50
trees = []

# Обучение ансамбля
for _ in range(n_trees):
    X_bs, y_bs = bootstrap_sample(X_train, y_train)
    # TODO: Создайте и обучите дерево на бутстрэп-выборке
    # tree = ...
    # 
    trees.append(tree)

# Предсказание и агрегация
predictions = np.array([tree.predict(X_test) for tree in trees])
mode_result = stats.mode(predictions, axis=0, keepdims=True)
bagging_pred = mode_result.mode[0]
bagging_acc = accuracy_score(y_test, bagging_pred)

print(f"Точность бэггинга (вручную): {bagging_acc:.3f}")